In [1]:
!git clone https://github.com/hassanaliemon/BanglaASR.git
%cd /content/BanglaASR

Cloning into 'BanglaASR'...
remote: Enumerating objects: 39, done.
remote: Counting objects: 100% (39/39), done.
remote: Compressing objects: 100% (36/36), done.
remote: Total 39 (delta 11), reused 26 (delta 3), pack-reused 0
Receiving objects: 100% (39/39), 579.54 KiB | 10.54 MiB/s, done.
Resolving deltas: 100% (11/11), done.
/content/BanglaASR


In [2]:
!git clone https://huggingface.co/hassanaliemon/BanglaASR
!mv BanglaASR saved_model

Cloning into 'BanglaASR'...
remote: Enumerating objects: 28, done.
remote: Counting objects: 100% (24/24), done.
remote: Compressing objects: 100% (24/24), done.
remote: Total 28 (delta 3), reused 0 (delta 0), pack-reused 4
Unpacking objects: 100% (28/28), 564.45 KiB | 5.88 MiB/s, done.
Filtering content: 100% (5/5), 2.70 GiB | 47.03 MiB/s, done.


In [3]:
!unzip /content/infer_mp3.zip

Archive:  /content/infer_mp3.zip
   creating: infer_mp3/
  inflating: infer_mp3/common_voice_bn_31255511.mp3  
  inflating: infer_mp3/common_voice_bn_31255513.mp3  
  inflating: infer_mp3/common_voice_bn_31255517.mp3  
  inflating: infer_mp3/common_voice_bn_31255520.mp3  
  inflating: infer_mp3/common_voice_bn_31255522.mp3  
  inflating: infer_mp3/common_voice_bn_31255548.mp3  
  inflating: infer_mp3/common_voice_bn_31255551.mp3  
  inflating: infer_mp3/common_voice_bn_31255554.mp3  
  inflating: infer_mp3/common_voice_bn_31255556.mp3  
  inflating: infer_mp3/common_voice_bn_31255557.mp3  


In [11]:
# !pip install evaluate
!pip install ipython

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 8.8 MB/s eta 0:00:00


In [13]:
import os
import librosa
import random
import torch
import evaluate
import torchaudio
import numpy as np
import pandas as pd
from datasets import Dataset
from dataclasses import dataclass
from typing import Any, Dict, List, Union
from matplotlib import pyplot as plt
from glob import glob
import IPython.display as ipd

from transformers import WhisperTokenizer, WhisperProcessor
from model.feature_extraction_whisper import WhisperFeatureExtractor
from model.modeling_whisper import WhisperForConditionalGeneration

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('device',device)

device cuda


In [6]:
audio_path = 'infer_mp3'
model_path = 'saved_model'

In [7]:
audio_files = glob(audio_path+'/*.mp3')
audio_files

['infer_mp3/common_voice_bn_31255513.mp3',
 'infer_mp3/common_voice_bn_31255517.mp3',
 'infer_mp3/common_voice_bn_31255520.mp3',
 'infer_mp3/common_voice_bn_31255554.mp3',
 'infer_mp3/common_voice_bn_31255556.mp3',
 'infer_mp3/common_voice_bn_31255551.mp3',
 'infer_mp3/common_voice_bn_31255548.mp3',
 'infer_mp3/common_voice_bn_31255522.mp3',
 'infer_mp3/common_voice_bn_31255511.mp3',
 'infer_mp3/common_voice_bn_31255557.mp3']

In [8]:
feature_extractor = WhisperFeatureExtractor.from_pretrained(model_path)
tokenizer = WhisperTokenizer.from_pretrained(model_path+'/tokenizer')
processor = WhisperProcessor.from_pretrained(model_path+'/tokenizer')
model = WhisperForConditionalGeneration.from_pretrained(model_path).to(device)


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [18]:
def prepare_dataset(path):
    speech_array, sampling_rate = torchaudio.load(path, format="mp3")
    speech_array = speech_array[0].numpy()
    speech_array = librosa.resample(np.asarray(speech_array), orig_sr=sampling_rate, target_sr=16000)
    input_features = feature_extractor(speech_array, sampling_rate=16000, return_tensors="pt").input_features
    return speech_array, input_features
def bn_asr(audio_file):
    _, input_features = prepare_dataset(audio_file)
    # np_feature = input_features[0].detach().cpu().numpy()
    predicted_ids = model.generate(inputs=input_features.to(device))[0]
    transcription = processor.decode(predicted_ids, skip_special_tokens=True)
    return transcription

In [17]:
ipd.Audio(audio_files[0])

In [19]:
print('transcription', bn_asr(audio_files[0]))

transcription দলে তিনি মূলতঃ ডানহাতি ব্যাটসম্যান হিসেবে খেলে থাকেন।


In [21]:
idx=1
ipd.Audio(audio_files[idx])

In [22]:
print('transcription', bn_asr(audio_files[idx]))

transcription তার নিজের লেখা গানটি নারীবাদের প্রতি সম্মাননা প্রদর্শন।


In [23]:
idx=2
ipd.Audio(audio_files[idx])

In [24]:
print('transcription', bn_asr(audio_files[idx]))

transcription ঐ সিরিজে চার টেস্টে অংশ নেয় তার দল।


In [25]:
idx=3
ipd.Audio(audio_files[idx])

In [26]:
print('transcription', bn_asr(audio_files[idx]))

transcription স্বস্তার বরফ জলে কি জল দেয়কে যানে।


In [27]:
idx=3
ipd.Audio(audio_files[idx])

In [28]:
print('transcription', bn_asr(audio_files[idx]))

transcription স্বস্তার বরফ জলে কি জল দেয়কে যানে।


In [31]:
idx=4
ipd.Audio(audio_files[idx])

In [32]:
print('transcription', bn_asr(audio_files[idx]))

transcription তিনি বাংলাদেশের জাতীজনক ও প্রয়াত রাষ্ট্রপতি শেখ মুজিবুর রহমানের ব্যক্তিগত চিকিৎসক হিসাবে দায়িত্ব পালন করেছেন।


In [33]:
idx=5
ipd.Audio(audio_files[idx])

In [34]:
print('transcription', bn_asr(audio_files[idx]))

transcription চাষের আওতাধীন এলাকা বাড়ার সাথে সাথে কৃষকরা সমবায় গঠন করেছিল যাতে তারা তাদের ফসলের ন্যায্যমূল্য পেতে পারে।


In [35]:
idx=6
ipd.Audio(audio_files[idx])

In [36]:
print('transcription', bn_asr(audio_files[idx]))

transcription নিয়মিত না হলেও তারা সংকর বন্ধুপাধ্যায় এই গোষ্ঠীর আলোচনায় মাঝে মাঝে যোগ দিতেন।


In [37]:
idx=7
ipd.Audio(audio_files[idx])

In [38]:
print('transcription', bn_asr(audio_files[idx]))

transcription আয়রন্ম্যান্ড পরবর্তী স্নায়জউদ্ধের বিষয়ে থেকে সমসাময়িক উদ্বেগ, যেমন কর্পোরেন্ট অপরাধ এবং সন্ত্রালবাদের বিরুদ্ধা যুদ্ধ শুরু করেন।


In [39]:
idx=8
ipd.Audio(audio_files[idx])

In [40]:
print('transcription', bn_asr(audio_files[idx]))

transcription তিনি প্রথম ব্যক্তি হন যে একই সময়ে দুইটি চ্যাম্পিয়ন ছিলেন।


In [43]:
idx=9
ipd.Audio(audio_files[idx])

In [44]:
print('transcription', bn_asr(audio_files[idx]))

transcription তিনি গদ্য ও কবিতা রচনায় ধ্রুপদী রুশ ঐতিহ্য ধারণের মাধ্যমে সভিল্পীকতায় অটল থাকার কারণে প্রসিদ্ধি লাভ করেন।
